# Udacity Project for P3: Open Street Data Wrangling

*Author: Jie Hu,   jie.hu.ds@gmail.com*

This is a project for Udacity [Nano-degree](https://classroom.udacity.com/nanodegrees/) P3 part. 
In this project I will clean the open street data of [San Francisco Bay Area](https://mapzen.com/data/metro-extracts/metro/san-francisco-bay_california/) and write into MongoDB database, after which yield some visualization work. 

### Step 1. Load Data and list problems

In [93]:
# Load packages and raw data file
import re
from collections import defaultdict
import json
import xml.etree.cElementTree as ET
import pprint
import numpy as np

osm_file = 'sf_sample.osm'

Print sample data from this file:

In [94]:
with open(osm_file, 'r') as f:
    for i in range(500):
        print f.readline()

<?xml version="1.0" encoding="UTF-8"?>

<osm>

  <node changeset="22950379" id="281266" lat="37.5601845" lon="-122.3025783" timestamp="2014-06-15T18:55:15Z" uid="1660455" user="juergenb22" version="6" />

	<node changeset="4472283" id="26117856" lat="37.5860936" lon="-122.3313774" timestamp="2010-04-19T20:46:46Z" uid="14293" user="KindredCoda" version="15" />

	<node changeset="36847187" id="26795963" lat="37.6097145" lon="-122.3916753" timestamp="2016-01-27T22:33:35Z" uid="2748195" user="karitotp" version="19" />

	<node changeset="10936318" id="29891422" lat="37.6897725" lon="-122.1017753" timestamp="2012-03-10T20:18:38Z" uid="482916" user="bill42" version="18" />

	<node changeset="36866799" id="29891439" lat="37.6378577" lon="-122.0980391" timestamp="2016-01-28T22:08:37Z" uid="2748195" user="karitotp" version="6">

		<tag k="ref" v="25B" />

		<tag k="source" v="survey;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/92.pdf" />

		<tag k="exit_to" v="Hesperian Boulevard" />


Check tags of sample of 100000 lines

In [72]:
dic = defaultdict(int)
line_num = 100000

for event, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    dic[elem.tag] +=1
    line_num -=1
dic

defaultdict(int, {'bounds': 1, 'node': 93198, 'tag': 6802})

In [96]:
# 'exit_to' can have multiple values, should create a list to involve them

for _, elem in ET.iterparse(osm_file):

    for tag in elem.iter('tag'):
        if tag.attrib['k'] == 'exit_to':
            print tag.attrib['v']


Hesperian Boulevard
Hesperian Boulevard
Stone Valley Road East
Stone Valley Road East
Jackson Street; San Mateo; San Mateo Bridge;CA-92
Jackson Street; San Mateo; San Mateo Bridge;CA-92
Redwood Road
Redwood Road
66th Street;Zhone Way
66th Street;Zhone Way
CA 13 South; Ashby Avenue
CA 13 South; Ashby Avenue
SR-84 East;Thornton Avenue
SR-84 East;Thornton Avenue
Broadway-Auto Row;Webster Street
Broadway-Auto Row;Webster Street
Potrero Avenue
Potrero Avenue
Acalanes Road;Upper Happy Valley Road
Acalanes Road;Upper Happy Valley Road
Paul Avenue
Paul Avenue
Daly City;I-280
Daly City;I-280
Fremont Street
Fremont Street
SR-92 West;Half Moon Bay;SR-92 East;Fashion Island Boulevard
SR-92 West;Half Moon Bay;SR-92 East;Fashion Island Boulevard
Hillsdale Boulevard;Foster City
Hillsdale Boulevard;Foster City
El Camino Real South
El Camino Real South
West Hillsdale Boulevard
West Hillsdale Boulevard
Whipple Avenue
Whipple Avenue
Buchanan Street; Albany
Buchanan Street; Albany
Central Avenue
Central A

In [97]:
# Node
line_num = 10000
for _, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    for node in elem.iter('node'):
        print node.attrib
    line_num -=1

{'changeset': '22950379', 'uid': '1660455', 'timestamp': '2014-06-15T18:55:15Z', 'lon': '-122.3025783', 'version': '6', 'user': 'juergenb22', 'lat': '37.5601845', 'id': '281266'}
{'changeset': '4472283', 'uid': '14293', 'timestamp': '2010-04-19T20:46:46Z', 'lon': '-122.3313774', 'version': '15', 'user': 'KindredCoda', 'lat': '37.5860936', 'id': '26117856'}
{'changeset': '36847187', 'uid': '2748195', 'timestamp': '2016-01-27T22:33:35Z', 'lon': '-122.3916753', 'version': '19', 'user': 'karitotp', 'lat': '37.6097145', 'id': '26795963'}
{'changeset': '10936318', 'uid': '482916', 'timestamp': '2012-03-10T20:18:38Z', 'lon': '-122.1017753', 'version': '18', 'user': 'bill42', 'lat': '37.6897725', 'id': '29891422'}
{'changeset': '36866799', 'uid': '2748195', 'timestamp': '2016-01-28T22:08:37Z', 'lon': '-122.0980391', 'version': '6', 'user': 'karitotp', 'lat': '37.6378577', 'id': '29891439'}
{'changeset': '5595644', 'uid': '14293', 'timestamp': '2010-08-26T04:41:03Z', 'lon': '-122.4035252', 'ver

In [78]:
'''
{'k': 'is_in:continent', 'v': 'North America'}
{'k': 'is_in:country', 'v': 'United States'}
{'k': 'is_in:country_code', 'v': 'US'}
should use a dict
'''

line_num = 10000
for _, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    for tag in elem.iter('tag'):
        print tag.attrib
    line_num -=1

{'k': 'ref', 'v': '19A'}
{'k': 'source', 'v': 'http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivenorth.pdf;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivesouth.pdf'}
{'k': 'exit_to', 'v': 'I-280'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'ref', 'v': '19A'}
{'k': 'source', 'v': 'http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivenorth.pdf;http://www.dot.ca.gov/hq/traffops/signtech/calnexus/pdf/eightyfivesouth.pdf'}
{'k': 'exit_to', 'v': 'I-280'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'ref', 'v': '433B'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'ref', 'v': '433B'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'ref', 'v': '436'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'ref', 'v': '436'}
{'k': 'highway', 'v': 'motorway_junction'}
{'k': 'name', 'v': 'Chase'}
{'k': 'amenity', 'v': 'bank'}
{'k': 'created_by', 'v': 'JOSM'}
{'k': 'name', 'v': 'Chase'}
{'k': 'amenity', 'v': 'bank'}
{'k': 'created_by', 'v': 'J

In [33]:
'Joaquin Miller Road; Lincoln Avenue'.split(';')   # should strip witin each item

['Joaquin Miller Road', ' Lincoln Avenue']

In [62]:
line_num = 10000
positions = []
for _, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    for node in elem.iter('node'):
        position = [node.attrib['lat'], node.attrib['lon']]
        if position not in positions:
            positions.append(position)
        else:
            print position
    line_num -=1

In [58]:
pos = np.array([[1,2],[3,4]])

In [79]:
# timestamp to year, month, day, time

In [82]:
sty = re.compile(r'.*:.*[:]?.*')
sty.findall('test')

[]

In [85]:
sty = re.compile(r'.*:.*[:]?.*')

line_num = 100000
for _, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    for tag in elem.iter('tag'):
        if tag.attrib['k'] != 'source':    
            arr = sty.findall(tag.attrib['k'])
            if arr != []:
                print tag.attrib
    line_num -=1

{'k': 'source:highway', 'v': 'survey'}
{'k': 'source:highway', 'v': 'survey'}
{'k': 'census:population', 'v': '2010'}
{'k': 'is_in:continent', 'v': 'North America'}
{'k': 'is_in:country', 'v': 'United States'}
{'k': 'is_in:country_code', 'v': 'US'}
{'k': 'name:de', 'v': 'San Francisco'}
{'k': 'name:eu', 'v': 'San Francisco'}
{'k': 'name:fa', 'v': u'\u0633\u0627\u0646 \u0641\u0631\u0627\u0646\u0633\u06cc\u0633\u06a9\u0648'}
{'k': 'name:hi', 'v': u'\u0938\u0948\u0928 \u092b\u094d\u0930\u093e\u0902\u0938\u093f\u0938\u094d\u0915\u094b'}
{'k': 'name:ja', 'v': u'\u30b5\u30f3\u30d5\u30e9\u30f3\u30b7\u30b9\u30b3'}
{'k': 'name:kn', 'v': u'\u0cb8\u0cbe\u0ca8\u0ccd \u0cab\u0ccd\u0cb0\u0cbe\u0ca8\u0ccd\u0cb8\u0cbf\u0cb8\u0ccd\u0c95\u0cca'}
{'k': 'name:ko', 'v': u'\uc0cc\ud504\ub780\uc2dc\uc2a4\ucf54'}
{'k': 'name:pt', 'v': u'S\xe3o Francisco'}
{'k': 'name:ru', 'v': u'\u0421\u0430\u043d-\u0424\u0440\u0430\u043d\u0446\u0438\u0441\u043a\u043e'}
{'k': 'name:ta', 'v': u'\u0bb8\u0bbe\u0ba9\u0bcd \u0baa\

In [89]:
# exam node
sty = re.compile(r'.*:.*[:]?.*')

line_num = 10000
for _, elem in ET.iterparse(osm_file):
    if line_num < 0:
        break;
    for node in elem.iter('node'):   
        for key in node.keys():
            if sty.match(key):
                print node.attrib
    line_num -=1

'Los Altos '